In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np

np.set_printoptions(precision=3)

In [104]:
# error reporting function
def create_error_dict(**kwargs):
	error_dict = {}
	
	for err_name, errs in kwargs.items():
		mean = np.mean(errs)
		max = np.max(errs)

		error_dict[f"{err_name}_mean"] = mean
		error_dict[f"{err_name}_max"] = max

	return error_dict

### Stability (and correctness) of Hessenberg-ification

In [108]:
from scipy.linalg import hessenberg
from utils import get_random_symmetric_matrix, make_upper_hessenberg

ZERO = 1e-15

def test_hessenberg(test_count = 100, size = 5, a = 0.0, b = 1.0):
	As = []
	errors = []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		As.append(A)

		Hp, _ = make_upper_hessenberg(A)
		H = hessenberg(A)

		errors.append(np.abs(Hp - H))

	return As, errors

In [107]:
As, errs = test_hessenberg(100, 200)
create_error_dict(mae=errs)

{'mae_mean': np.float64(2.888161030635708e-15),
 'mae_max': np.float64(1.432921781230334e-09)}

### Correctness of Hessenberg QR

In [106]:
from utils import hessenberg_qr

ZERO = 1e-15

def test_hessenberg_qr(test_count = 100, size = 5, a = 0.0, b = 1.0):
	As = []
	recon_errors = []
	q_errors = []
	r_errors = []

	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		As.append(A)

		H, _ = make_upper_hessenberg(A)
		Qp, Rp = hessenberg_qr(H)
		Q, R = np.linalg.qr(H)

		recon_errors.append(np.abs(Qp @ Rp - H))
		q_errors.append(np.abs(Q - Qp))
		r_errors.append(np.abs(R - Rp))

	return As, recon_errors, q_errors, r_errors


In [109]:
As, recon_errors, q_errors, r_errors = test_hessenberg_qr(100, 500)
create_error_dict(reconstruction=recon_errors, q_mae=q_errors, r_mae=r_errors)

{'reconstruction_mean': np.float64(7.953248318450292e-17),
 'reconstruction_max': np.float64(1.545430450278218e-13),
 'q_mae_mean': np.float64(1.7880189072632124e-16),
 'q_mae_max': np.float64(6.54309939562836e-13),
 'r_mae_mean': np.float64(3.1794146452756553e-16),
 'r_mae_max': np.float64(9.002798506685394e-13)}

### Practical-QR Step

In [25]:
from eigen import practical_qr

In [56]:
A = get_random_symmetric_matrix(500)

In [57]:
e1 = practical_qr(A)

In [132]:
def test_qr_iteration(test_count = 100, size = 5, a = 0.0, b = 1.0):
	errors = []
	
	for t in range(test_count):
		A = get_random_symmetric_matrix(size, a, b)
		As.append(A)

		evals, evecs = practical_qr(A)

		err = 0.0
		for j, eval in enumerate(evals):
			vec = evecs[:, j]

			err += np.mean(np.abs(A @ vec - eval * vec))
		
		errors.append(err / len(evals))

	return errors

In [133]:
errs = test_qr_iteration(100, 50)
create_error_dict(evec=errs)

{'evec_mean': np.float64(1.2981901996682546e-11),
 'evec_max': np.float64(2.330312905538164e-11)}

### SVD

In [134]:
from svd import svd

In [205]:
A = np.random.uniform(0.0, 1.0, (27, 12))

In [204]:
u1, s1, v1 = np.linalg.svd(A, full_matrices=False)

In [203]:
u2, s2, v2 = svd(A)